
# University Accountability Ordinance (UAO) — Midterm Notebook
**Course:** CS506 Data Science (Spring 2025)  
**Client:** Councilor Liz Breadon’s Office  
**Notebook generated:** 2025-10-27 22:08

This notebook is the working artifact for the **midterm deliverables**:
- preliminary visualizations (EDA),
- detailed data processing description (with executable steps),
- modeling methods prototyped so far,
- preliminary results.

> Tip: Keep cells concise and add short markdown notes above each cell describing why the step is needed and what changed.


## 1. Setup & Configuration

In [8]:
!which python
!!pip install numpy pandas matplotlib scikit-learn --quiet


/Users/berkkom/Desktop/university_accountablilty_ordinance/.venv/bin/python


[]

In [9]:

# ==== Environment Checks & Imports ====
# If a package import fails, install it locally (e.g., `pip install pandas scikit-learn pyproj shapely`).
# Keep your environment pinned via requirements.txt for reproducibility.

import sys
import os
import math
import json
import pathlib
from typing import List, Dict, Tuple

# Core DS stack
import numpy as np
import pandas as pd

# Viz (follow course rules: use matplotlib for charts)
import matplotlib.pyplot as plt

# Modeling
try:
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, classification_report
except Exception as e:
    print('scikit-learn not available yet. Install it if you plan to run the modeling cells.')
    print(e)

# GIS helpers (optional, comment out if unavailable)
try:
    import geopandas as gpd
except Exception as e:
    print('geopandas not available (optional).')
    print(e)

# Paths
PROJECT_ROOT = pathlib.Path('.').resolve()
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
CLEAN_DIR = DATA_DIR / 'cleaned'
VIZ_DIR = PROJECT_ROOT / 'visualizations'

for d in [DATA_DIR, RAW_DIR, CLEAN_DIR, VIZ_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print('Project root:', PROJECT_ROOT)
print('Data directories created (if missing).')


geopandas not available (optional).
No module named 'geopandas'
Project root: /Users/berkkom/Desktop/university_accountablilty_ordinance/notebooks
Data directories created (if missing).


## 2. Project Constants & Links

In [16]:

# === Official data sources (update if your team keeps copied snapshots in Drive) ===
URLS = {
    "violations": "https://data.boston.gov/api/3/action/datastore_search?resource_id=800a2663-1d6a-46e7-9356-bedb70f5332c&limit=50000",
    "requests_311": "https://data.boston.gov/api/3/action/datastore_search?resource_id=2968e2c0-d479-49ba-a884-4ef523ada3c0&limit=50000",  # check resource_id for 311
    "sam_addresses": "https://data.boston.gov/api/3/action/datastore_search?resource_id=4d01b43b-49f2-4e56-bc1b-cb7738eae6b2&limit=50000", # verify latest SAM resource
    "assessments": "https://data.boston.gov/api/3/action/datastore_search?resource_id=0625b6ca-7f79-4e0f-bb55-9d4a63f877f8&limit=50000",   # property assessment example id
    # Student Housing Reports (UAO): typically provided via client/drive — place CSVs in data/raw/uao/
}

DATA_FILES = {
    "violations": RAW_DIR / "violations_2016_2024.csv",
    # Add others when ready:
    # "311": RAW_DIR / "requests311_2016_2024.csv",
    # "assessments": RAW_DIR / "assessments_2024.csv",
}

# Years of interest
YEARS = list(range(2016, 2025))
print("Configured local data files:")
for name, path in DATA_FILES.items():
    print(f" - {name}: {path}")
print('Configured data URLs (verify resource_ids in your environment).')


Configured local data files:
 - violations: /Users/berkkom/Desktop/university_accountablilty_ordinance/notebooks/data/raw/violations_2016_2024.csv
Configured data URLs (verify resource_ids in your environment).


## 3. Data Acquisition

In [ ]:

# === Downloader utilities (CKAN-style API) ===
import urllib.request
import urllib.error

def fetch_ckan_json(url: str) -> dict:
    try:
        with urllib.request.urlopen(url) as resp:
            return json.loads(resp.read().decode('utf-8'))
    except urllib.error.URLError as e:
        print('Fetch failed:', url, e)
        return {}

def records_to_df(payload: dict, key: str = 'result') -> pd.DataFrame:
    if not payload:
        return pd.DataFrame()
    result = payload.get(key, {})
    recs = result.get('records', [])
    return pd.DataFrame(recs)

# Example: pull a small sample to get schema; for full pulls consider paging (offset/limit).
violations_sample = records_to_df(fetch_ckan_json(URLS['violations']))
print('Violations sample shape:', violations_sample.shape)
violations_sample.head()



## 4. Load Local Snapshots (Preferred for Reproducibility)
Place raw CSV extracts in `data/raw/` first (e.g., exported from the portal or provided by the client).
Name suggestions (adjust to your files):
- `violations_2016_2024.csv`
- `requests311_2016_2024.csv`
- `assessments_2024.csv`
- `sam_addresses.csv`
- `uao_students_2016_2024.csv`


In [20]:
from pathlib import Path

print("RAW_DIR:", RAW_DIR)
print("Files in RAW_DIR:")
for p in sorted(RAW_DIR.glob("*")):
    print(" -", p.name)

RAW_DIR: /Users/berkkom/Desktop/university_accountablilty_ordinance/notebooks/data/raw
Files in RAW_DIR:
 - violations_2016_2024.csv


In [23]:

# Template loaders — update filenames to your actual snapshots
def load_csv_safe(path: pathlib.Path, **kwargs) -> pd.DataFrame:
    if not path.exists():
        print(f'[WARN] Missing file: {path}')
        return pd.DataFrame()
    try:
        df = pd.read_csv(path, **kwargs)
        print(f'[OK] Loaded {path.name}:', df.shape)
        return df
    except Exception as e:
        print(f'[ERR] Could not load {path}:', e)
        return pd.DataFrame()

df_viol = load_csv_safe(RAW_DIR / 'violations_2016_2024.csv', low_memory=False)
df_311  = load_csv_safe(RAW_DIR / 'requests311_2016_2024.csv', low_memory=False)
df_assess = load_csv_safe(RAW_DIR / 'assessments_2024.csv', low_memory=False)
df_sam = load_csv_safe(RAW_DIR / 'sam_addresses.csv', low_memory=False)
df_uao = load_csv_safe(RAW_DIR / 'uao_students_2016_2024.csv', low_memory=False)

df_viol.head()
df_viol.info()



[OK] Loaded violations_2016_2024.csv: (16953, 25)
[WARN] Missing file: /Users/berkkom/Desktop/university_accountablilty_ordinance/notebooks/data/raw/requests311_2016_2024.csv
[WARN] Missing file: /Users/berkkom/Desktop/university_accountablilty_ordinance/notebooks/data/raw/assessments_2024.csv
[WARN] Missing file: /Users/berkkom/Desktop/university_accountablilty_ordinance/notebooks/data/raw/sam_addresses.csv
[WARN] Missing file: /Users/berkkom/Desktop/university_accountablilty_ordinance/notebooks/data/raw/uao_students_2016_2024.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16953 entries, 0 to 16952
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   _id               16953 non-null  int64  
 1   case_no           16953 non-null  object 
 2   ap_case_defn_key  16953 non-null  int64  
 3   status_dttm       16952 non-null  object 
 4   status            16953 non-null  object 
 5   code              1

## 5. Cleaning & Standardization

In [ ]:

# Address normalization helpers
import re

def normalize_address(s: str) -> str:
    if not isinstance(s, str):
        return ''
    s = s.upper().strip()
    s = re.sub(r'\s+', ' ', s)
    # common street suffix normalizations (expand as needed)
    s = s.replace(' STREET', ' ST').replace(' AVENUE', ' AVE').replace(' ROAD', ' RD')
    s = s.replace(' PLACE', ' PL').replace(' COURT', ' CT').replace(' BOULEVARD', ' BLVD')
    return s

# Example cleaning pipeline for violations
def clean_violations(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df

    out = df.copy()
    out.columns = [c.strip().lower() for c in out.columns]

    # --- Parse date ---
    out["status_dttm"] = pd.to_datetime(out["status_dttm"], errors="coerce")
    out["year"] = out["status_dttm"].dt.year

    # --- Build a single normalized address string ---
    out["full_address"] = (
        out["violation_stno"].astype(str).fillna("") + " " +
        out["violation_street"].astype(str).fillna("") + " " +
        out["violation_suffix"].astype(str).fillna("") + ", " +
        out["violation_city"].astype(str).fillna("") + ", " +
        out["violation_state"].astype(str).fillna("") + " " +
        out["violation_zip"].astype(str).fillna("")
    )
    out["full_address"] = out["full_address"].str.replace(r"\s+", " ", regex=True).str.strip().str.upper()

    # --- Add a simplified severity label ---
    def classify_severity(text):
        if not isinstance(text, str):
            return "other"
        text = text.upper()
        if any(k in text for k in ["SMOKE", "STRUCT", "ELECTRICAL", "FIRE", "CO", "HEAT", "HAZARD"]):
            return "severe"
        elif any(k in text for k in ["SANIT", "PEST", "TRASH", "PLUMBING"]):
            return "moderate"
        else:
            return "minor"

    out["severity"] = out["description"].map(classify_severity)

    # --- Drop duplicates and irrelevant columns (optional) ---
    out = out.drop_duplicates(subset=["case_no"])
    return out


def clean_311(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    out.columns = [c.strip().lower().replace(' ', '_') for c in out.columns]
    for c in [col for col in ['open_dt','closed_dt','target_dt','on_time_dt'] if col in out.columns]:
        out[c] = pd.to_datetime(out[c], errors='coerce')
    # Filter to relevant categories; adjust to match your schema
    if 'subject' in out.columns:
        mask = out['subject'].str.contains('HOUS|SANIT|NOISE|HEAT|WATER', case=False, na=False)
        out = out[mask]
    return out

def clean_assess(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    out.columns = [c.strip().lower().replace(' ', '_') for c in out.columns]
    # Numeric conversions
    for c in ['total_value','land_value','building_value','year_built','living_area']:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors='coerce')
    return out

df_viol_c = clean_violations(df_viol)
df_311_c  = clean_311(df_311)
df_assess_c = clean_assess(df_assess)

for name, df in [('violations', df_viol_c), ('311', df_311_c), ('assess', df_assess_c)]:
    print(name, df.shape)


## 6. Linking Datasets via Address / SAM ID

In [ ]:

# Expectation: SAM dataset includes a stable address identifier (e.g., 'SAM_ID' or similar).
# Adjust these keys to your actual column names.

def prepare_sam(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    out.columns = [c.strip().lower().replace(' ', '_') for c in out.columns]
    # Example expected columns; update for your schema:
    # 'sam_id', 'full_address', 'x', 'y', 'latitude', 'longitude'
    if 'full_address' in out.columns:
        out['full_address_norm'] = out['full_address'].map(normalize_address)
    return out

df_sam_c = prepare_sam(df_sam)

# Example: join violations to SAM by normalized address
def link_to_sam(df_any: pd.DataFrame, df_sam_c: pd.DataFrame, address_col_guess: List[str]) -> pd.DataFrame:
    if df_any.empty or df_sam_c.empty:
        return df_any
    any_c = df_any.copy()
    for col in address_col_guess:
        if col in any_c.columns:
            any_c[col + '_norm'] = any_c[col].map(normalize_address)
            # left join on normalized address
            any_c = any_c.merge(
                df_sam_c[['full_address_norm','sam_id']] if 'sam_id' in df_sam_c.columns else df_sam_c[['full_address_norm']],
                left_on=col + '_norm',
                right_on='full_address_norm',
                how='left'
            )
            break
    return any_c

df_viol_linked = link_to_sam(df_viol_c, df_sam_c, address_col_guess=['address','full_address','location'])
df_311_linked  = link_to_sam(df_311_c, df_sam_c, address_col_guess=['address','full_address','location'])

print('Linked violations shape:', df_viol_linked.shape)
print('Linked 311 shape:', df_311_linked.shape)


## 7. Student Presence Flags (UAO merge)

In [ ]:

# Expectation: df_uao has columns like ['address','year','student_count'] per address/year.
# Adjust to your schema and join key.

def prepare_uao(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    out.columns = [c.strip().lower().replace(' ', '_') for c in out.columns]
    if 'address' in out.columns:
        out['address_norm'] = out['address'].map(normalize_address)
    return out

df_uao_c = prepare_uao(df_uao)

def tag_student_addresses(df_any: pd.DataFrame, df_uao_c: pd.DataFrame, by_year=True) -> pd.DataFrame:
    if df_any.empty or df_uao_c.empty:
        return df_any
    out = df_any.copy()
    # join key: normalized address (+ year if available)
    if by_year and 'year' in out.columns and 'year' in df_uao_c.columns:
        out = out.merge(
            df_uao_c[['address_norm','year','student_count']],
            left_on=['address_norm','year'] if 'address_norm' in out.columns else ['address_norm','year'],  # adjust if necessary
            right_on=['address_norm','year'],
            how='left'
        )
    else:
        out = out.merge(
            df_uao_c[['address_norm','student_count']],
            on='address_norm',
            how='left'
        )
    out['is_student_address'] = out['student_count'].fillna(0) > 0
    return out

# Apply only if your cleaned frames have 'address_norm'
if 'address_norm' in df_viol_linked.columns:
    df_viol_tagged = tag_student_addresses(df_viol_linked, df_uao_c)
else:
    df_viol_tagged = df_viol_linked

print('Student-tagged violations shape:', df_viol_tagged.shape)


## 8. Preliminary Visualizations (EDA)

In [ ]:

# 8.1 Violation counts by year (student-linked vs others)
if not df_viol_tagged.empty and 'year' in df_viol_tagged.columns:
    vc = (
        df_viol_tagged
        .assign(student=lambda d: np.where(d['is_student_address'], 'student', 'non-student') if 'is_student_address' in d.columns else 'unknown')
        .groupby(['year','student'])
        .size()
        .reset_index(name='count')
        .pivot(index='year', columns='student', values='count')
        .fillna(0)
    )
    ax = vc.plot(kind='line', marker='o', figsize=(8,5))  # matplotlib is used under the hood
    ax.set_title('Violation Counts by Year (Student vs Non-Student)')
    ax.set_xlabel('Year'); ax.set_ylabel('Count')
    plt.tight_layout()
    plt.show()
else:
    print('Skip 8.1: df_viol_tagged empty or missing year.')


In [ ]:

# 8.2 Top violation types among student-linked properties
if not df_viol_tagged.empty:
    # Guess a violation description column; update as needed
    viol_col = 'viol_desc' if 'viol_desc' in df_viol_tagged.columns else None
    if viol_col:
        top = (
            df_viol_tagged[df_viol_tagged.get('is_student_address', False) == True]
            .groupby(viol_col)
            .size()
            .sort_values(ascending=False)
            .head(15)
        )
        ax = top.plot(kind='bar', figsize=(9,5))
        ax.set_title('Top Violation Types (Student-Linked)')
        ax.set_xlabel('Violation'); ax.set_ylabel('Count')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print('Skip 8.2: violation description column not found.')
else:
    print('Skip 8.2: df_viol_tagged empty.')


In [ ]:

# 8.3 Property value distributions: student vs non-student (join assess + student tags by address/SAM)
if not df_assess_c.empty and not df_viol_tagged.empty:
    # Build a list of known student addresses from UAO
    student_addrs = set(df_uao_c['address_norm']) if not df_uao_c.empty and 'address_norm' in df_uao_c.columns else set()
    df_assess_c['address_norm'] = df_assess_c.get('address', '').map(lambda x: x.upper().strip() if isinstance(x,str) else '')
    df_assess_c['is_student_address'] = df_assess_c['address_norm'].isin(student_addrs)
    # Choose a value column
    val_col = 'total_value' if 'total_value' in df_assess_c.columns else None
    if val_col:
        grouped = df_assess_c.groupby('is_student_address')[val_col].describe()
        print(grouped)
        # Simple boxplot
        ax = df_assess_c.boxplot(column=val_col, by='is_student_address', figsize=(7,5))
        plt.title('Property Values by Student Address Flag'); plt.suptitle('')
        plt.xlabel('Is Student Address'); plt.ylabel('Assessed Value')
        plt.tight_layout()
        plt.show()
    else:
        print('Skip 8.3: total_value column not found in assessments.')
else:
    print('Skip 8.3: missing assess or violations data.')


## 9. Modeling Prototype — Predicting Non-Compliance

In [ ]:

# Define non-compliance label: >=3 violations within a 24-month window (example)
def label_non_compliance(df: pd.DataFrame, window_days: int = 730) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    # Ensure date exists
    date_col = None
    for c in ['date','open_dt','issued_date','status_date']:
        if c in out.columns:
            date_col = c; break
    if not date_col:
        print('No date column to compute windows.')
        out['non_compliant'] = np.nan
        return out
    out = out.sort_values([date_col])
    # Group by address (or SAM ID if available)
    key = 'sam_id' if 'sam_id' in out.columns else ('address_norm' if 'address_norm' in out.columns else None)
    if not key:
        print('No join key (sam_id/address_norm); labeling by global sequence.')
        key = None
    labels = []
    if key:
        for addr, grp in out.groupby(key):
            dates = grp[date_col].dropna().sort_values().values
            # Sliding window over sorted dates
            count_label = 0
            if len(dates) >= 3:
                i = 0
                for j in range(len(dates)):
                    while dates[j] - dates[i] > np.timedelta64(window_days, 'D'):
                        i += 1
                    if (j - i + 1) >= 3:
                        count_label = 1
                        break
            labels.extend([count_label]*len(grp))
    else:
        labels = [0]*len(out)
    out['non_compliant'] = labels if labels else 0
    return out

df_for_model = label_non_compliance(df_viol_tagged)

# Feature engineering (very light; extend for final)
def build_features(df_viol: pd.DataFrame, df_311: pd.DataFrame, df_assess: pd.DataFrame) -> pd.DataFrame:
    if df_viol.empty:
        return pd.DataFrame()
    key = 'sam_id' if 'sam_id' in df_viol.columns else ('address_norm' if 'address_norm' in df_viol.columns else None)
    if not key:
        print('No key for features.')
        return pd.DataFrame()
    # Violations per address
    f_viol = df_viol.groupby(key).agg(
        viol_count=('non_compliant', 'size'),
        severe_count=('severity', lambda s: (s=='severe').sum() if 'severe' in s.values or True else 0),
        non_compliant_any=('non_compliant', 'max')
    ).reset_index()
    # 311 per address (optional)
    if not df_311.empty:
        if key in df_311.columns:
            f_311 = df_311.groupby(key).size().reset_index(name='req311_count')
            f_viol = f_viol.merge(f_311, on=key, how='left')
        else:
            f_viol['req311_count'] = 0
    else:
        f_viol['req311_count'] = 0
    # Assessment merge
    if not df_assess.empty and key in df_assess.columns:
        cols = [c for c in ['total_value','year_built','living_area'] if c in df_assess.columns]
        f_viol = f_viol.merge(df_assess[[key] + cols], on=key, how='left')
    # Fill NaNs
    f_viol = f_viol.fillna({'req311_count': 0})
    return f_viol

features = build_features(df_for_model, df_311_linked, df_assess_c)
print('Feature table shape:', features.shape)
features.head()


In [ ]:

# Train a simple logistic regression (if sklearn is available and enough data)
try:
    target = 'non_compliant_any'
    feature_cols = [c for c in ['viol_count','severe_count','req311_count','total_value','year_built','living_area'] if c in features.columns]
    if len(feature_cols) >= 2 and target in features.columns:
        model_df = features.dropna(subset=feature_cols + [target]).copy()
        X = model_df[feature_cols].values
        y = model_df[target].values.astype(int)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
        scaler = StandardScaler()
        X_train_s = scaler.fit_transform(X_train)
        X_test_s = scaler.transform(X_test)
        clf = LogisticRegression(max_iter=200)
        clf.fit(X_train_s, y_train)
        y_proba = clf.predict_proba(X_test_s)[:,1]
        auc = roc_auc_score(y_test, y_proba)
        print('Logistic Regression AUROC:', round(auc, 3))
    else:
        print('Not enough feature columns or missing target; skip modeling.')
except Exception as e:
    print('Modeling step skipped due to error:', e)


## 10. Preliminary Results (Fill in as you run cells)


- **Coverage:** XX% of violation records matched to SAM / addresses.
- **Top violation categories:** (list from Section 8.2)
- **Temporal trend:** (brief 1–2 sentence insight from 8.1)
- **Value comparison:** (insight from 8.3 boxplot)
- **Modeling:** Logistic regression AUROC = `X.XX` (features: ...). Notes on what improves/harms performance.

> Save figures to `visualizations/` and reference them in your README midterm report.


## 11. Utility: Save Figures

In [ ]:

def save_current_fig(name: str, directory: pathlib.Path = VIZ_DIR):
    directory.mkdir(parents=True, exist_ok=True)
    outpath = directory / name
    plt.savefig(outpath, bbox_inches='tight', dpi=180)
    print('Saved:', outpath)


## 12. Appendix — Reproducibility Notes


- Keep raw data immutable in `data/raw/`.  
- Output cleaned tables to `data/cleaned/` with versioned filenames.  
- Track your environment in `requirements.txt` (pin to minor versions).  
- Document any manual data fixes in a `CHANGELOG.md` with rationale and links.  
- Consider adding a `Makefile` or `invoke` tasks to orchestrate end-to-end runs.
